In [1]:
# There must be a bug in Vscode notebook make me have to import this first, otherwise I get a modulenotfounderror
from fastai.callback.core import *

from IPython.core.debugger import set_trace as bk
import os
from pathlib import Path
from functools import partial
import torch
import nlp
from transformers import ElectraTokenizerFast
hf_tokenizer = ElectraTokenizerFast.from_pretrained("google/electra-small-generator")
from hugdatafast import *

# 1.Basics

## Simple tokenization & infer cache name
`cols`(`Dict[str]`): tokenize the every column named key into column named its value  
`cols`(`List[str]`): specify the name of columns to be tokenized, replace the original columns' data with tokenized one

Here, we tokenized "sentence" into a new column named "text_idxs", the "sentence" column still exist.

In [2]:
cola = nlp.load_dataset('glue', 'cola')
tokenized_cola = cola.my_map(simple_tokenize_func({'sentence':'text_idxs'}, hf_tokenizer),
                             cache_file_names='tokenized_{split}')
print(tokenized_cola['train'][0])
print()
for dset in tokenized_cola.values(): print(dset.cache_files[0]['filename'])

{'idx': 0, 'label': 1, 'sentence': "Our friends won't buy this analysis, let alone the next one we propose.", 'text_idxs': [2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012]}

/home/yisiang/.cache/huggingface/datasets/glue/cola/1.0.0/637080968c182118f006d3ea39dd9937940e81cfffc8d79836eaae8bba307fc4/tokenized_train.arrow
/home/yisiang/.cache/huggingface/datasets/glue/cola/1.0.0/637080968c182118f006d3ea39dd9937940e81cfffc8d79836eaae8bba307fc4/tokenized_validation.arrow
/home/yisiang/.cache/huggingface/datasets/glue/cola/1.0.0/637080968c182118f006d3ea39dd9937940e81cfffc8d79836eaae8bba307fc4/tokenized_test.arrow


## Create fastai `Dataloaders` and `show_batch`

`cols`: **specify columns whose values form a output sample in order**, and the semantic type of each column to encode/decode, with one of the following signature (see doc).

Here, `['text_idxs, 'label']` is equal to `{'text_idxs': TensorText, 'label': TensorCategory}`

The bars are sorting samples according to length, see `MySortedDL`

In [3]:
cola_dsets = HF_Datasets(tokenized_cola, cols=['text_idxs', 'label'], hf_toker=hf_tokenizer, neat_show=True)
cola_dls = cola_dsets.dataloaders(bs=32)
cola_dls.show_batch(max_n=2) # show at most two rows

98%|█████████▊| 1040/1063 [00:01<00:00, 588.25it/s]

,text_idxs,label
0,"everybody who has ever, worked in any office which contained any typewriter which had ever been used to type any letters which had to be signed by any administrator who ever worked in any department like mine will know what i mean.",1
1,"i finally worked up enough courage to ask which people up at corporate headquarters the sooner i solve this problem, the quicker i'll get free of.",0


You can either specify `neat_show=False` (which is default), to show real data which is tokenized and  with pad 

In [4]:
cola_dsets = HF_Datasets(tokenized_cola, cols={'text_idxs': TensorText, 'label': TensorCategory}, hf_toker=hf_tokenizer)
cola_dls = cola_dsets.dataloaders(bs=32)
cola_dls.show_batch(max_n=2)

98%|█████████▊| 1042/1063 [00:02<00:00, 397.20it/s]

,text_idxs,label
0,"everybody who has ever , worked in any office which contained any type ##writer which had ever been used to type any letters which had to be signed by any administrator who ever worked in any department like mine will know what i mean .",1
1,the prisoner must have been being inter ##rogated when the supervisor walked into the room and saw what was going on and put a stop to it . [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD],1


`test_with_label` is `False` by default, so in test set the sample formed by only first `n_inp` columns specified, which is x.

This make you able to apply the same to all splits when test set come with no y or fake y

## Multiple columns (> 2) in sample
Some points to notice:
- title of each column showed is and in order of `cols` specified in `HF_Datasets`
- auto pad sequence to the max length in the batch, for all columns
- If a fastai semantic tensor type is not specified, it look dtype and shape of the tensor and decide how to decode it autmatically 

In [5]:
wsc = nlp.load_dataset('super_glue', 'wsc.fixed')
print(wsc['train'][0])
tokenized_wsc = wsc.my_map(simple_tokenize_func(['text', 'span1_text', 'span2_text'], hf_tokenizer))
wsc_dsets = HF_Datasets(tokenized_wsc, cols={'text': TensorText, 'span1_index': noop, 'span1_text':TensorText, 'span2_index': noop, 'span2_text': TensorText, 'label': lambda t: t.bool()}, # convert label (int) to (bool), just to test its abililty to show tensor(bool)
hf_toker=hf_tokenizer)
dls = wsc_dsets.dataloaders(bs=3, srtkey_fc=False, shuffle_train=False) # don't sort samples, don't shuffle trainset
#bk()
dls.show_batch()

{'idx': 0, 'label': 0, 'span1_index': 0, 'span1_text': 'Mark', 'span2_index': 13, 'span2_text': 'He', 'text': 'Mark told Pete many lies about himself, which Pete included in his book. He should have been more skeptical.'}


,text,span1_index,span1_text,span2_index,span2_text,label
0,"mark told pete many lies about himself , which pete included in his book . he should have been more skeptical . [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]",0,mark [PAD] [PAD],13,he,0
1,"the mothers of arthur and celeste have come to the town to fetch them . they are very happy to have them back , but they sc ##old them just the same because they ran away . [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]",1,mothers [PAD] [PAD],25,them,0
2,"mark was close to mr . singer ' s heels . he heard him calling for the captain , promising him , in the jar ##gon everyone talked that night , that not one thing should be damaged on the ship except only the ammunition , but the captain and all his crew had best stay in the cabin until the work was over",4,mr . singer,8,he,0


# 2. Aggregate Dataset
a sample in transformed dataset is aggregated/accumulated from multiple original samples.

- Except for `LMTransform`, you can implement your own logic create a class inherits `AggregateTransform` and implements `accumulate` and `create_example` method

- Note that you should pass **tokenized** dataset(s)

## Make  dataset for (traditional) language model
You can always pass dict of `nlp.Dataset` or a `nlp.Dataset` at your will for any transform class, we've test passing a dict, now we test a `nlp.Dataset`

In [6]:
cola_val = tokenized_cola['validation']
lm_cola_val = LMTransform(cola_val, max_len=20, text_col='text_idxs').map()

print('Original dataset:')
print('num of samples:', len(cola['validation']))
print('second to last sentence:', cola['validation'][-2]['sentence'])
print('          last sentence:', cola['validation'][-1]['sentence'])
print('LM dataset:')
print('num of sampels:', len(lm_cola_val))
assert len(lm_cola_val) == 481
print('last text (x):', hf_tokenizer.decode(lm_cola_val[-1]['x_text']))
print('last text (y):', hf_tokenizer.decode(lm_cola_val[-1]['y_text']))

Original dataset:
num of samples: 1043
second to last sentence: John arranged for himself to get the prize.
          last sentence: John talked to Bill about himself.
LM dataset:
num of sampels: 481
last text (x): . john talked to bill about himself
last text (y): john talked to bill about himself.


In [7]:
lm_cola = LMTransform(tokenized_cola, max_len=50, text_col='text_idxs').map()
# test single dataset
lm_ds = HF_Dataset(lm_cola['validation'], cols={'x_text':LMTensorText, 'y_text':TensorText},hf_toker=hf_tokenizer)
lm_dl = MySortedDL(lm_ds, srtkey_fc=False)
lm_dl.show_batch(max_n=2)

,x_text,y_text
0,"the sailors rode the breeze clear of the rocks . the weights made the rope stretch over the pull ##ey . the mechanical doll wr ##ig ##gled itself loose . if you had eaten more , you would want less . as you eat the most , you want the","sailors rode the breeze clear of the rocks . the weights made the rope stretch over the pull ##ey . the mechanical doll wr ##ig ##gled itself loose . if you had eaten more , you would want less . as you eat the most , you want the least"
1,". the more you would want , the less you would eat . i demand that the more john eat , the more he pays . mary listen ##s to the grateful dead , she gets depressed . the ang ##rier mary got , the more she looked at pictures","the more you would want , the less you would eat . i demand that the more john eat , the more he pays . mary listen ##s to the grateful dead , she gets depressed . the ang ##rier mary got , the more she looked at pictures ."


## Test ELECTRA data creating

In [8]:
proc_dset = ELECTRADataTransform(cola['validation'], is_docs=False, text_col='sentence', max_length=128, hf_toker=hf_tokenizer).map()
e_dsets = HF_Datasets({'train':proc_dset}, cols={'input_ids':TensorText,'sentA_lenth':noop}, hf_toker=hf_tokenizer)
e_dls = e_dsets.dataloaders(srtkey_fc=False)
e_dls.show_batch(max_n=2)

,input_ids,sentA_lenth
0,[CLS] herman whipped the sugar and the cream . linda taped the picture to the wall . linda taped the picture onto the wall . the child and her mother clung together . this flyer and that flyer differ . this flyer and that flyer differ apart . the jewel ##ler sc ##ri ##bbled the contract with his name . the gardener grew that ac ##orn into an oak tree . [SEP] i shaped a loaf . the children amused . susan whispered the news . susan whispered at rachel . ellen said that mel ##ons were selling well . ellen said about the present conditions . ellen warned helen against skating on thin ice . cynthia ni ##bbled on the carrot . cynthia chewed . paul [SEP],72
1,"[CLS] the mayor regarded as being absurd the proposal to build a sidewalk from dartmouth to smith . i want that bill left to remain a secret . i know a man who tom drives as drives . drowning cats , which is against the law , are hard to rescue . himself is understood by rutherford . [SEP] muriel said nothing else than that she had been insulted . i feel that arch will show up . the proof this set is rec ##urs ##ive is difficult . the mad ##ri ##gal ##s which henry plays the lu ##te and sings sound lou ##sy . tom picked these grapes , and i washed some turn ##ip ##s , and su ##zie will prepare these grapes . [SEP]",59


# 3. Test filtering feature
Note that filter won't be applied to split other than train, because validation/test set is for fair comparison, and you can't take out samples at your will 

In [9]:
l = 23
num = {}
for split in tokenized_cola:
  num[split] = reduce(lambda sum, sample: sum+(1 if len(sample['text_idxs'])==l else 0), 
                      tokenized_cola[split], 0)
print(num)

{'train': 26, 'validation': 2, 'test': 6}


In [10]:
ccola_dsets = HF_Datasets(tokenized_cola, cols=['text_idxs', 'label'], hf_toker=hf_tokenizer)
ccola_dls = ccola_dsets.dataloaders(filter_fc=lambda text_idxs, label: len(text_idxs)!=l,)

for i, split in enumerate(tokenized_cola):
  if split == 'train':
    assert ccola_dls[i].n == len(tokenized_cola[split])-num[split],f"{split}: filtered: {ccola_dls[i].n}, unfiltered: {len(tokenized_cola[split])}, should be filtered: {num[split]}"
  else:
    assert ccola_dls[i].n == len(tokenized_cola[split]), f"{split}: accidentally filtered: {ccola_dls[i].n}, unfiltered: {len(tokenized_cola[split])}"
print("Test passed")

96%|█████████▋| 1025/1063 [00:02<00:00, 380.90it/s]Test passed


# 4. Cache dataloader
If sorting or filtering is applied, dataloader need to create some record inside it, to do it only once, we can cache the records. 

If `cache_dir` is not specified, it will be the cache_dir of `dsets` passed to `HF_Datasets`.

In [11]:
for f in ['/tmp/cached_train.json','/tmp/cached_val.json', '/tmp/cached_test.json']:
  if Path(f).exists(): os.remove(f)

ccola_dls = ccola_dsets.dataloaders(cache_dir='/tmp', cache_name='cached_{split}.json')

99%|█████████▊| 1049/1063 [00:02<00:00, 388.21it/s]

This time we load the caches, it should be fast and progress bars sholdn't appear

In [12]:
ccola_dls = ccola_dsets.dataloaders(cache_dir='/tmp', cache_name='cached_{split}.json')